Alberto Moro

Instalaciones rqueridas hasta la fecha (Reinstalar ctransformer usando Metal.)

In [ ]:
!pip install llama-cpp-python
!pip install numpy
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
!CMAKE_ARGS="-DLLAMA_METAL_EMBED_LIBRARY=ON -DLLAMA_METAL=on" pip install -U llama-cpp-python --no-cache-dir
!pip uninstall ctransformers --yes
!CT_METAL=1 pip install ctransformers --no-binary ctransformers
#!conda install -c pytorch faiss-gpu
!pip install faiss-cpu

In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Creamos Callback

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

Comprobar si se está usando la GPU del M2.

In [8]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


Intento en GPU

(Recordar) Instalación con METAL para permitir uso de hilos de GPU en M2.

Cargamos el llm de LLama2 con GPU.

In [43]:
# Callbacks support token-wise streaming
callback_manager2 = CallbackManager([StreamingStdOutCallbackHandler()])
                                    
n_gpu_layers = 40  # Mayor valor implica más uso del chip, evitar que Metal haga deallocating porque nos estamos pasando y funcionará mal, también varia según tamaño de prompt.
n_batch = 30  # Cantidad de tokens que se mete al modelo de golpe, afecta al uso de memoria.

#En definitiva jugar con los dos valores hasta que se use un batch que no exceda la memoria pero tampoco sea muy grande para la cpu usada y lo mismo con las layers, que no saturen la cpu ni memoria.

llmGPU = LlamaCpp(
    #model_path="/Users/albertomoro/Desktop/UC/LangChain/Pruebas/llama-2-13b-chat.Q2_K.gguf",llama-2-7b.Q2_K.gguf
    model_path="/Users/albertomoro/Desktop/UC/LangChain/Pruebas/llama-2-7b-chat.Q2_K.gguf", #Modelo más pequeño disponible.
    input={"temperature": 0.75, "max_length": 20, "top_p": 1},
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager2,
    verbose=True,
)

/Users/albertomoro/anaconda3/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/albertomoro/Desktop/UC/LangChain/Pruebas/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                       

Cargamos documentos que se vayan a pasar

Ecoratings

In [48]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("ecorating.pdf")
data = loader.load()

Opcion 1: Separar PDF en chunks por parrafos

In [52]:
from langchain.text_splitter import CharacterTextSplitter

# split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=400, chunk_overlap=20)
documents = text_splitter.split_documents(data)
# print the number of chunks obtained
len(documents)
#text_chunks

339

Crear embeddings del texto de los PDFs para que lo use el modelo

In [51]:
#Cargamos modelo de embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Creamos vectorstore (dos alternativas para probar cual va mejor)

Opcion 2 (Recomendada por mi hasta el momento)

In [65]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings
    )
vector_store.persist()


Creamos el pipeline con langchain

Preguntas a la LLM Chain con GPU.

Ejemplo de prompt:

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

Mejor ejemplo hasta la fecha:

Cadena de Retrieval QA con Prompt a medida.

In [61]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Keep the answer as concise as possible, use less than 20 words. Always say "Alberto me controla" at the end of the answer. The next info is the most updated about the question and is true, more valid than any knowledge you previously had, use it for your answer:
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llmGPU,
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

Ejemplos de preguntas:

In [62]:
question = "What is the ecorating of Samsung Galaxy"
recuperado = vector_store.similarity_search(question)
print(recuperado)
print("Arriba, docs recuperados, abajo se mostrará la respuesta del LLM:")
result = qa_chain({"query": question})
result["result"]

[Document(page_content='The Samsung Galaxy A02s SM-A025G has an eco rating of 77, durability of 48, repairability of 66, recyclability of 77, climate efficiency of 54, resource efficiency of 85, RAM of 3 GB, and storage of 32 GB.', metadata={'page': 7, 'source': 'ecorating.pdf'}), Document(page_content='The Samsung Galaxy A03s SM-A037G has an eco rating of 79, durability of 58, repairability of 66, recyclability of 77, climate efficiency of 59, resource efficiency of 87, RAM of 3 GB, and storage of 32 GB.', metadata={'page': 7, 'source': 'ecorating.pdf'}), Document(page_content='The Samsung Galaxy A03 SM-A035G has an eco rating of 78, durability of 43, repairability of 51, recyclability of 63, climate efficiency of 57, resource efficiency of 86, RAM of 4 GB, and storage of 64 GB.', metadata={'page': 7, 'source': 'ecorating.pdf'}), Document(page_content='The Samsung Galaxy A03 SM-A035G has an eco rating of 78, durability of 43, repairability of 51, recyclability of 63, climate efficienc

Llama.generate: prefix-match hit


 Alberto me controla! The Samsung Galaxy A02s SM-A025G has an eco rating of 77, durability of 48, repairability of 66, recyclability of 77, climate efficiency of 54, resource efficiency of 85, RAM of 3 GB, and storage of 32 GB. Alberto me controla! The Samsung Galaxy A03s SM-A037G has an eco


llama_print_timings:        load time =    8183.69 ms
llama_print_timings:      sample time =      29.69 ms /   109 runs   (    0.27 ms per token,  3671.02 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   10334.64 ms /   109 runs   (   94.81 ms per token,    10.55 tokens per second)
llama_print_timings:       total time =   10924.83 ms /   110 tokens


' Alberto me controla! The Samsung Galaxy A02s SM-A025G has an eco rating of 77, durability of 48, repairability of 66, recyclability of 77, climate efficiency of 54, resource efficiency of 85, RAM of 3 GB, and storage of 32 GB. Alberto me controla! The Samsung Galaxy A03s SM-A037G has an eco'

In [59]:
question = "What is the phone with most ecorating?"
recuperado = vector_store.similarity_search(question)
print(recuperado)
print("Prueba")
result = qa_chain({"query": question})
result["result"]

[Document(page_content='The Deutsche Telekom AG T Phone S98317 has an eco rating of 74, durability of 54, repairability of 52, recyclability of 65, climate efficiency of 49, resource efficiency of 84, RAM of 4 GB, and storage of 64 GB.', metadata={'page': 2, 'source': 'ecorating.pdf'}), Document(page_content='The Deutsche Telekom AG T Phone Pro S98215 has an eco rating of 68, durability of 54, repairability of 52, recyclability of 65, climate efficiency of 36, resource efficiency of 80, RAM of 6 GB, and storage of 128 GB.', metadata={'page': 2, 'source': 'ecorating.pdf'}), Document(page_content='The NOTHING Phone (1) A063 has an eco rating of 77, durability of 50, repairability of 64, recyclability of 64, climate efficiency of 56, resource efficiency of 85, RAM of 12 GB, and storage of 256 GB.', metadata={'page': 1, 'source': 'ecorating.pdf'}), Document(page_content='The NOTHING Phone (1) A063 has an eco rating of 77, durability of 50, repairability of 64, recyclability of 64, climate 

Llama.generate: prefix-match hit


 Alberto me controla! The phone with the highest eco rating among those listed is the NOTHING Phone (1) A063 with an eco rating of 77.


llama_print_timings:        load time =    8183.69 ms
llama_print_timings:      sample time =      11.61 ms /    40 runs   (    0.29 ms per token,  3446.49 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    4844.86 ms /    40 runs   (  121.12 ms per token,     8.26 tokens per second)
llama_print_timings:       total time =    5074.26 ms /    41 tokens


' Alberto me controla! The phone with the highest eco rating among those listed is the NOTHING Phone (1) A063 with an eco rating of 77.'